<a href="https://colab.research.google.com/github/rks16/Collab-practice/blob/main/Project_fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset details:
* id : which is unique for each model
* title: title of news article
* author:
* text: the text of the article which could be incomplete
* label: either real or fake

0 - real news

1 - fake news

In [2]:
#importing dependencies

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
print(stopwords.words('english'))
c = len(stopwords.words('english'))
print('\n',c)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
#data collection and data preprocessing
news_data = pd.read_csv('/content/train.csv')


In [6]:
#dataframe shape rows x column
news_data.shape

(20800, 5)

In [7]:
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
#checking if all values are filled or is there any null
news_data.isnull()
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# replacing empty spaces in the dataframe with null string
news_data = news_data.fillna('')

in our project we need both title and author and combinne them so that it would be easy for us to predict

we dont use text data cuz its so big and is in paragraph and takes huge time for processing

In [10]:
#merging title and author column
news_data['content'] = news_data['author'] + ' ' + news_data['title']

print(news_data['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [11]:
#seperating data and label
X = news_data['content']
Y = news_data['label']


In [43]:
print(Y)

[1 0 1 ... 0 1 1]


---
Stemming:

It is the process of reducing a word to its root word such as prefixes and suffixes

eg:

actor,actress, acting --> they all have the root word 'act'

this helps in reducing the words and improve the performance of the model


---
After this process we vectorize the words..ie. converting words to their feature words (numerical data)


In [20]:
port_stem = PorterStemmer()

In [18]:
def stemming(content):
  stem_content = re.sub('[^a-zA-Z]',' ',content) # removing everything which is not alphabet like numbers and subst comma and fullstop by space ''
  stem_content = stem_content.lower() #converting all characters to lowercase
  stem_content = stem_content.split() #splitting words to lists
  # each word is stemmed (converted to its root word) and removing stopwords
  stem_content = [port_stem.stem(word) for word in stem_content if not word in stopwords.words('english')]
  stem_content = ' '.join(stem_content)

  return stem_content

  # basically this function
  # 1.remove everything except alphabets like numbers,punctuations
  # 2. convert everything to lower case letters
  # 3. split dataset to list
  # 4. stem in the dataset and remove all stopwords
  # 5. join all the words back to stem_content

In [22]:
news_data['content'] = news_data['content'].apply(stemming)

In [23]:
print(news_data['content'])

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [34]:
#seperating values
X = news_data['content'].values
Y = news_data['label'].values

In [35]:
# converting textual data to numerical vector data

vectorizer = TfidfVectorizer() # tf - term frequency idf - inverse doc freq
                               # tf - essentialy this counts the number of repeating words and assigs a numerical value to it understanding it to be a important word
                               # idf - removes which repeats many times and  are insignificant
vectorizer.fit(X)

X = vectorizer.transform(X)


In [36]:
print(X)

  (0, 15553)	0.28485063562728646
  (0, 13348)	0.2565896679337957
  (0, 8822)	0.3635963806326075
  (0, 8546)	0.29212514087043684
  (0, 7612)	0.24785219520671603
  (0, 6930)	0.21874169089359144
  (0, 4917)	0.233316966909351
  (0, 3759)	0.2705332480845492
  (0, 3567)	0.3598939188262559
  (0, 2933)	0.2468450128533713
  (0, 2464)	0.3676519686797209
  (0, 263)	0.27010124977708766
  (1, 16656)	0.3025156488372128
  (1, 6744)	0.19152496072048605
  (1, 5440)	0.7186013955384664
  (1, 3535)	0.2653147533915268
  (1, 2790)	0.19208753385709676
  (1, 2206)	0.36915639258038363
  (1, 1877)	0.15614790568229528
  (1, 1481)	0.2957471154505952
  (2, 15478)	0.41544962664721613
  (2, 9532)	0.49351492943649944
  (2, 5903)	0.3474613386728292
  (2, 5326)	0.3866530551182615
  (2, 3072)	0.46097489583229645
  :	:
  (20797, 12998)	0.2483705036831893
  (20797, 12227)	0.27276402145717243
  (20797, 12022)	0.24790022252744132
  (20797, 10216)	0.0804189541935242
  (20797, 9500)	0.17463635692029988
  (20797, 9431)	0.29394

Splitting data into trainning and test data

In [37]:
# splitting data

X_train,X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state=2, stratify=Y)

In [38]:
model = LogisticRegression()

model.fit(X_train ,Y_train)

LogisticRegression()

In [40]:
# evaluation
X_pre = model.predict(X_train)
accu_1 = accuracy_score(X_pre,Y_train)

print("The accuracy is : ", accu_1)

The accuracy is :  0.9866586538461538


In [41]:
# evaluation
X_pred = model.predict(X_test)
accu_2 = accuracy_score(X_pred,Y_test)

print("The accuracy is : ", accu_2)

The accuracy is :  0.9790865384615385


In [42]:
#predictive system

testing_content = X_test[4] #fetching data from test data

prediction = model.predict(testing_content)

print(prediction)

[0]


In [45]:
print(Y_test[4])

0
